# Surfingcrypto `Portfolio`

The `surfingcrypto` portfolio.

In [1]:
%load_ext autoreload
%autoreload 2
from surfingcrypto import Config
from surfingcrypto.portfolio import Portfolio
from surfingcrypto.portfolio.tracker import Tracker

import pandas as pd
c=Config("/Users/giorgiocaizzi/Documents/GitHub/surfingcrypto/config")

In [2]:
p=Portfolio("coinbase",configuration=c)

Warning! There are 5 transactionsthat were EXCLUDED in std_df.
Coinbase errors:
TransactionsHistory(Transactions:186 - Processed:186, Unhandled:0 - Errors:9)


In [3]:
p.std_df.head(10)

,type,amount,symbol,native_amount,nat_symbol,total,subtotal,total_fee,spot_price,trade_id
datetime,,,,,,,,,,
2018-10-22T06:29:22Z,buy,0.017237,BTC,100.00,EUR,100.00,97.01,2.99,5627.999719,None
2018-10-22T06:29:22Z,buy,100.000000,EUR,100.00,EUR,NaN,NaN,NaN,1.000000,None
2018-11-09T12:05:12Z,sell,0.017237,BTC,93.27,EUR,93.27,96.26,2.99,5584.488743,None
2018-11-09T12:05:13Z,sell,93.270000,EUR,93.27,EUR,NaN,NaN,NaN,1.000000,None
2018-11-09T12:06:14Z,buy,348.021666,BAT,93.27,EUR,93.27,90.28,2.99,0.259409,None
2018-11-09T12:06:15Z,buy,93.270000,EUR,93.27,EUR,NaN,NaN,NaN,1.000000,None
2019-01-25T14:28:56Z,sell,35.960000,EUR,35.96,EUR,NaN,NaN,NaN,1.000000,None
2019-01-25T14:28:56Z,sell,348.021666,BAT,35.96,EUR,35.96,37.95,1.99,0.109045,None
2021-04-20T18:57:40Z,buy,0.258429,ETH,500.00,EUR,500.00,492.66,7.34,1906.364408,None


In [4]:
p.std_df.total_fee.sum()

234.18000000000026

## Portfolio live value tracker

In [5]:
t=Tracker(p.std_df,stocks_start="1-1-2021",configuration=c)

In [6]:
t.stocks_start

Timestamp('2021-01-01 00:00:00+0000', tz='UTC')

In [7]:
t.portfolio_df

,Open date,Type,Qty,Symbol,Adj cost,Adj cost per share
0,2021-04-20 18:57:40+00:00,buy,0.258429,ETH,500.00,1906.364408
1,2021-04-22 10:13:53+00:00,buy,0.062128,MKR,250.00,3964.873854
2,2021-04-22 19:20:15+00:00,sell,0.062128,MKR,213.69,3491.496921
3,2021-04-24 08:53:47+00:00,buy,210.343863,ADA,213.69,1.000980
4,2021-04-25 16:15:33+00:00,buy,0.350116,CGLD,1.65,4.712730
...,...,...,...,...,...,...
163,2022-01-02 23:24:25+00:00,sell,1.026156,SOL,159.04,154.986191
164,2022-01-04 10:33:27+00:00,buy,0.067396,ETH,225.88,3351.534216
165,2022-01-04 10:44:49+00:00,buy,1.511428,SOL,225.73,149.348800
166,2022-01-04 10:44:51+00:00,sell,0.068600,ETH,230.19,3355.559414


In [8]:
print(len(t.portfolio_df.Symbol.unique()))

27


In [9]:
closedata=t.load_data()
closedata

,Date,Close,symbol
0,2021-01-01 00:00:00+00:00,600.859522,ETH
1,2021-01-02 00:00:00+00:00,638.265609,ETH
2,2021-01-03 00:00:00+00:00,796.329350,ETH
3,2021-01-04 00:00:00+00:00,848.990363,ETH
4,2021-01-05 00:00:00+00:00,894.422659,ETH
...,...,...,...
3029,2022-03-07 00:00:00+00:00,75.582331,SOL
3030,2022-03-08 00:00:00+00:00,75.478203,SOL
3031,2022-03-09 00:00:00+00:00,79.651404,SOL
3032,2022-03-10 00:00:00+00:00,75.308279,SOL


In [10]:
t.error_log

[]

In [11]:
print(len(closedata.symbol.unique()))

27


In [12]:
benchmark=t.set_benchmark("ETH")
benchmark

,Date,Close
0,2021-01-01 00:00:00+00:00,600.859522
1,2021-01-02 00:00:00+00:00,638.265609
2,2021-01-03 00:00:00+00:00,796.329350
3,2021-01-04 00:00:00+00:00,848.990363
4,2021-01-05 00:00:00+00:00,894.422659
...,...,...
430,2022-03-07 00:00:00+00:00,2298.639000
431,2022-03-08 00:00:00+00:00,2364.462260
432,2022-03-09 00:00:00+00:00,2465.147372
433,2022-03-10 00:00:00+00:00,2368.094996


In [13]:
active_portfolio=t.portfolio_start_balance()
active_portfolio

,Open date,Type,Qty,Symbol,Adj cost,Adj cost per share
0,2021-04-20 18:57:40+00:00,buy,0.258429,ETH,500.00,1906.364408
1,2021-04-22 10:13:53+00:00,buy,0.062128,MKR,250.00,3964.873854
2,2021-04-22 19:20:15+00:00,sell,0.062128,MKR,213.69,3491.496921
3,2021-04-24 08:53:47+00:00,buy,210.343863,ADA,213.69,1.000980
4,2021-04-25 16:15:33+00:00,buy,0.350116,CGLD,1.65,4.712730
...,...,...,...,...,...,...
163,2022-01-02 23:24:25+00:00,sell,1.026156,SOL,159.04,154.986191
164,2022-01-04 10:33:27+00:00,buy,0.067396,ETH,225.88,3351.534216
165,2022-01-04 10:44:49+00:00,buy,1.511428,SOL,225.73,149.348800
166,2022-01-04 10:44:51+00:00,sell,0.068600,ETH,230.19,3355.559414


In [14]:
active_portfolio.Type.unique()

array(['buy', 'sell'], dtype=object)

In [15]:
print(len(active_portfolio.Symbol.unique()))

27


In [37]:
positions_per_day=t.time_fill()

In [38]:
pd.set_option('display.max_columns', 500)


In [41]:
positions_per_day[]

,Open date,Type,Qty,Symbol,Adj cost,Adj cost per share,Date Snapshot
0,2021-04-20 18:57:40+00:00,buy,0.258429,ETH,500.0,1906.364408,2022-03-10 00:00:00+00:00


In [19]:
combined_df=t.per_day_portfolio_calcs(
    positions_per_day,
    benchmark,
    closedata,
    t.stocks_start 
)
combined_df

/opt/miniconda3/envs/cryptoenv/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Adj cost,Adj cost per share,Date Snapshot,Open date,Qty,Symbol,Type,Symbol Adj Close,Adj cost daily,Benchmark Close,Benchmark End Date Close,Benchmark Start Date Close,symbol End Date Close,symbol Start Date Close,Equiv Benchmark Shares,Benchmark Start Date Cost,Benchmark Return,symbol Return,symbol Share Value,Benchmark Share Value,Stock Gain / (Loss),Benchmark Gain / (Loss),Abs Value Compare,Abs Value Return,Abs. Return Compare
0,492.66,1906.364408,2021-04-21 00:00:00+00:00,2021-04-20 18:57:40+00:00,0.258429,ETH,buy,1964.593249,507.708026,1964.593249,2345.209878,600.859522,2345.209878,600.859522,0.819925,492.66,2.269638,0.030544,507.708026,1610.819957,15.048026,1118.159957,15.048026,0.030544,-2.239094
1,492.66,1906.364408,2021-04-22 00:00:00+00:00,2021-04-20 18:57:40+00:00,0.258429,ETH,buy,2000.536891,516.996908,2000.536891,2345.209878,600.859522,2345.209878,600.859522,0.819925,492.66,2.329459,0.049399,516.996908,1640.291064,24.336908,1147.631064,24.336908,0.049399,-2.280060
2,492.66,1906.364408,2021-04-23 00:00:00+00:00,2021-04-20 18:57:40+00:00,0.258429,ETH,buy,1953.619811,504.872170,1953.619811,2345.209878,600.859522,2345.209878,600.859522,0.819925,492.66,2.251375,0.024788,504.872170,1601.822557,12.212170,1109.162557,12.212170,0.024788,-2.226587
3,492.66,1906.364408,2021-04-24 00:00:00+00:00,2021-04-20 18:57:40+00:00,0.258429,ETH,buy,1828.220570,472.465360,1828.220570,2345.209878,600.859522,2345.209878,600.859522,0.819925,492.66,2.042676,-0.040991,472.465360,1499.004530,-20.194640,1006.344530,-20.194640,-0.040991,-2.083667
4,492.66,1906.364408,2021-04-25 00:00:00+00:00,2021-04-20 18:57:40+00:00,0.258429,ETH,buy,1915.747276,495.084806,1915.747276,2345.209878,600.859522,2345.209878,600.859522,0.819925,492.66,2.188345,0.004922,495.084806,1570.769902,2.424806,1078.109902,2.424806,0.004922,-2.183423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8127,225.73,149.348800,2022-03-09 00:00:00+00:00,2022-01-04 10:44:49+00:00,1.511428,SOL,buy,79.651404,120.387384,2465.147372,2345.209878,600.859522,73.982926,1.515448,0.375678,225.73,3.102702,-0.466675,120.387384,926.102851,-105.342616,700.372851,-105.342616,-0.466675,-3.569377
8128,186.85,182.087335,2022-03-10 00:00:00+00:00,2021-11-29 10:47:17+00:00,1.026156,SOL,buy,75.308279,77.278038,2368.094996,2345.209878,600.859522,73.982926,1.515448,0.310971,186.85,2.941179,-0.586417,77.278038,736.409316,-109.571962,549.559316,-109.571962,-0.586417,-3.527596
8129,225.73,149.348800,2022-03-10 00:00:00+00:00,2022-01-04 10:44:49+00:00,1.511428,SOL,buy,75.308279,113.823063,2368.094996,2345.209878,600.859522,73.982926,1.515448,0.375678,225.73,2.941179,-0.495756,113.823063,889.642360,-111.906937,663.912360,-111.906937,-0.495756,-3.436935
8130,186.85,182.087335,2022-03-11 00:00:00+00:00,2021-11-29 10:47:17+00:00,1.026156,SOL,buy,73.982926,75.918019,2345.209878,2345.209878,600.859522,73.982926,1.515448,0.310971,186.85,2.903092,-0.593695,75.918019,729.292704,-110.931981,542.442704,-110.931981,-0.593695,-3.496787


In [20]:
combined_df.columns

Index(['Adj cost', 'Adj cost per share', 'Date Snapshot', 'Open date', 'Qty',
       'Symbol', 'Type', 'Symbol Adj Close', 'Adj cost daily',
       'Benchmark Close', 'Benchmark End Date Close',
       'Benchmark Start Date Close', 'symbol End Date Close',
       'symbol Start Date Close', 'Equiv Benchmark Shares',
       'Benchmark Start Date Cost', 'Benchmark Return', 'symbol Return',
       'symbol Share Value', 'Benchmark Share Value', 'Stock Gain / (Loss)',
       'Benchmark Gain / (Loss)', 'Abs Value Compare', 'Abs Value Return',
       'Abs. Return Compare'],
      dtype='object')

In [21]:
combined_df["Date Snapshot"].unique()

<DatetimeArray>
['2021-04-21 00:00:00+00:00', '2021-04-22 00:00:00+00:00',
 '2021-04-23 00:00:00+00:00', '2021-04-24 00:00:00+00:00',
 '2021-04-25 00:00:00+00:00', '2021-04-26 00:00:00+00:00',
 '2021-04-27 00:00:00+00:00', '2021-04-28 00:00:00+00:00',
 '2021-04-29 00:00:00+00:00', '2021-04-30 00:00:00+00:00',
 ...
 '2022-03-02 00:00:00+00:00', '2022-03-03 00:00:00+00:00',
 '2022-03-04 00:00:00+00:00', '2022-03-05 00:00:00+00:00',
 '2022-03-06 00:00:00+00:00', '2022-03-07 00:00:00+00:00',
 '2022-03-08 00:00:00+00:00', '2022-03-09 00:00:00+00:00',
 '2022-03-10 00:00:00+00:00', '2022-03-11 00:00:00+00:00']
Length: 325, dtype: datetime64[ns, UTC]

In [22]:
combined_df.groupby([combined_df["Date Snapshot"].dt.date]).sum()

,Adj cost,Adj cost per share,Qty,Symbol Adj Close,Adj cost daily,Benchmark Close,Benchmark End Date Close,Benchmark Start Date Close,symbol End Date Close,symbol Start Date Close,Equiv Benchmark Shares,Benchmark Start Date Cost,Benchmark Return,symbol Return,symbol Share Value,Benchmark Share Value,Stock Gain / (Loss),Benchmark Gain / (Loss),Abs Value Compare,Abs Value Return,Abs. Return Compare
Date Snapshot,,,,,,,,,,,,,,,,,,,,,
2021-04-21,492.66,1906.364408,0.258429,1964.593249,507.708026,1964.593249,2345.209878,600.859522,2345.209878,600.859522,0.819925,492.66,2.269638,0.030544,507.708026,1610.819957,15.048026,1118.159957,15.048026,0.030544,-2.239094
2021-04-22,492.66,1906.364408,0.258429,2000.536891,516.996908,2000.536891,2345.209878,600.859522,2345.209878,600.859522,0.819925,492.66,2.329459,0.049399,516.996908,1640.291064,24.336908,1147.631064,24.336908,0.049399,-2.280060
2021-04-23,492.66,1906.364408,0.258429,1953.619811,504.872170,1953.619811,2345.209878,600.859522,2345.209878,600.859522,0.819925,492.66,2.251375,0.024788,504.872170,1601.822557,12.212170,1109.162557,12.212170,0.024788,-2.226587
2021-04-24,492.66,1906.364408,0.258429,1828.220570,472.465360,1828.220570,2345.209878,600.859522,2345.209878,600.859522,0.819925,492.66,2.042676,-0.040991,472.465360,1499.004530,-20.194640,1006.344530,-20.194640,-0.040991,-2.083667
2021-04-25,703.21,1907.365388,210.602292,1916.647431,684.426877,3831.494551,4690.419755,1201.719044,2345.933833,601.003779,1.170340,703.21,4.376689,-0.095804,684.426877,2242.075880,-18.783123,1538.865880,-18.783123,-0.095804,-4.472494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-07,18494.36,47308.706666,7122.889196,36945.927973,17436.225077,73556.447989,75046.716082,19227.504708,37687.671625,9617.377207,30.779840,18494.36,90.418711,0.091213,17436.225077,70751.740803,-1058.134923,52257.380803,-1058.134923,0.091213,-90.327498
2022-03-08,18494.36,47308.706666,7122.889196,37999.146640,17822.597063,75662.792311,75046.716082,19227.504708,37687.671625,9617.377207,30.779840,18494.36,93.924263,0.756144,17822.597063,72777.770221,-671.762937,54283.410221,-671.762937,0.756144,-93.168119
2022-03-09,18494.36,47308.706666,7122.889196,39618.929775,18443.713265,78884.715908,75046.716082,19227.504708,37687.671625,9617.377207,30.779840,18494.36,99.286454,1.879016,18443.713265,75876.841880,-50.646735,57382.481880,-50.646735,1.879016,-97.407437


In [26]:
t.line(combined_df, ["Stock Gain / (Loss)"])
# t.line_facets(combined_df, 'symbol Return', 'Benchmark Return')